In [ ]:
from depmapomics.config import *
from depmap_omics_upload import tracker
from depmap_omics_upload import upload
from depmapomics import mutations
from genepy.utils import helper as h

In [ ]:
from depmap_omics_upload import tracker, upload
mytracker = tracker.SampleTracker()

In [ ]:
upload.initVirtualDatasets(samplesetname="22Q4")

In [ ]:
from depmap_omics_upload import tracker, upload

virtual = {'internal': '766e45863f8549feafda32aab71a9555',
 'dmc': '033e4a5eea94476c8832addaa0532e2a',
 'public': '7aa37823a3ed4c2ba211e75fb3d4157e'}

In [ ]:
upload.makeModelLvMatrices(virtual_ids=virtual, files_nummat=LATEST2FN_NUMMAT_MODEL, files_table=LATEST2FN_TABLE_MODEL)

In [ ]:
upload.makePRLvMatrices(virtual_ids=virtual, files_nummat=LATEST2FN_NUMMAT_PR, files_table=LATEST2FN_TABLE_PR)

In [ ]:
upload.uploadAuxTables(taiga_ids=virtual_dryrun,)

In [ ]:
set(CCLE_expression.index) - set(OmicsExpressionProteinCodingGenesTPMLogp1.index)

In [ ]:
shared_cols = [c in CCLE_expression.columns if c in OmicsExpressionProteinCodingGenesTPMLogp1.columns]
CCLE_expression = CCLE_expression[shared_cols]

In [ ]:
CCLE_expression.corrwith(OmicsExpressionProteinCodingGenesTPMLogp1)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsSomaticMutations = tc.get(name='internal-22q2dryrun-5ffb', version=39, file='OmicsSomaticMutations')
OmicsSomaticMutations.columns

In [ ]:
OmicsSomaticMutations

In [ ]:
O_micsStructuralVariantsProfile = tc.get(name='dmc-22q2dryrun-0f8d', version=28, file='O​micsStructuralVariantsProfile')


In [ ]:
O_micsStructuralVariantsProfile

In [ ]:
OmicsLoH = tc.get(name='dmc-22q2dryrun-0f8d', file='OmicsLoH')
OmicsLoH

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

somaticMutations_profile = tc.get(name='mutations-latest-f263', version=8, file='somaticMutations_profile')

In [ ]:
somaticMutations_model = somaticMutations_profile[
            somaticMutations_profile["DepMap_ID"].isin(set(pr2model_dict.keys()))
        ].replace({"DepMap_ID": pr2model_dict})
somaticMutations_model.to_csv("somaticMutations_model.csv", index=False)

In [ ]:
tc.update_dataset(
    changes_description="add model level maf",
    dataset_permaname="mutations-latest-f263",
    upload_files=[            
        {
            "path": "somaticMutations_model.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
d = upload.getPRToRelease()

In [ ]:
default_table = upload.makeDefaultModelTable(d['internal'])
pr2model_dict = dict(list(zip(default_table.ProfileID, default_table.ModelID)))

In [ ]:
pr2model_dict

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()

In [ ]:
pr_table = mytracker.read_pr_table()
seq_table = mytracker.add_model_cols_to_seqtable(cols=["ModelID"])

In [ ]:
pr_table[pr_table.ModelCondition.isnull()]

In [ ]:
pr_table.ModelCondition

In [ ]:
import pandas as pd

for pr in pr_table.index:
    if (pr_table.loc[pr, "Datatype"] == "wgs" or pr_table.loc[pr, "Datatype"] == "wes") and not (pd.isna(pr_table.loc[pr, "MainSequencingID"])):
        cdsid = pr_table.loc[pr, "MainSequencingID"]
        if seq_table.loc[cdsid, "source"] == 'CCLE2':
            if seq_table.loc[cdsid, "ModelID"] not in OmicsCNGene.index:
                print(seq_table.loc[cdsid, "ModelID"])

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
OmicsCNGene = tc.get(name='internal-22q2dryrun-5ffb', version=86, file='OmicsCNSegmentsProfile')


In [ ]:
# grant gs access
import dalmatian as dm
ws = dm.WorkspaceManager("fccredits-silver-tan-7621/CCLE_v2").disable_hound()
ccle2samples = ws.get_samples()

In [ ]:
cols = ['hg38_rna_bam', 'hg38_rna_bai', 'hg38_wgs_bam', 'hg38_wgs_bai', 'hg38_wes_bam', 'hg38_wes_bai']

In [ ]:
from genepy.utils import helper as h
import pandas as pd

cmds = []
for i in ccle2samples.index:
    for c in ['hg38_wgs_bam', 'hg38_wgs_bai']:
        if pd.isna(ccle2samples.loc[i, c]) == False:
            if '/wgs/' in ccle2samples.loc[i, c] or '/wes/' in ccle2samples.loc[i, c]:
                print(ccle2samples.loc[i, c])
            cmds.append("gsutil -u broad-firecloud-ccle -m acl ch -R -g All:R " + ccle2samples.loc[i, c])

In [ ]:
h.parrun(cmds, cores=8)

In [ ]:
cmds

In [ ]:
old_maf = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/submissions/cfeeb3ac-7eaa-48ac-8afa-8ba6316e9c93/run_vcf_to_depmap/ea5d21ec-b350-4ee2-a391-9d26e21ce96b/call-vcf_to_depmap/attempt-2/CDS-0Z9yIY-maf-coding_somatic-subset.csv.gz")

In [ ]:
old_maf

In [ ]:
new_maf = pd.read_csv("gs://fc-secure-9dffc819-20a8-49ea-8fa8-1b1bab1475d0/submissions/58b92d2c-7d5e-483c-a7d9-ca3a952458d2/WGS_pipeline/5be4117d-bc64-4e51-aca4-c30dca3d170b/call-my_vcf_to_depmap/CDS-0Z9yIY-maf-coding_somatic-subset.csv.gz")

In [ ]:
new = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/submissions/745d8e9e-5651-453e-842a-7add2f3c16b5/run_vcf_to_depmap/29fa5ba6-df81-416e-9768-5fe13a605a99/call-vcf_to_depmap/attempt-4/CDS-00Nrci-maf-coding_somatic-subset.csv.gz")
old = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/submissions/cfeeb3ac-7eaa-48ac-8afa-8ba6316e9c93/run_vcf_to_depmap/3d2f9581-58e4-4337-84f4-ce21866a596a/call-vcf_to_depmap/CDS-00Nrci-maf-coding_somatic-subset.csv.gz")

In [ ]:
new.cosmic_hotspot.value_counts()

In [ ]:
old.compare(new)

In [ ]:
from genepy.utils import helper as h
h.parrun(cmds, cores=8)
print("done")

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsSomaticMutationsMatrixDamaging = tc.get(name='internal-22q2dryrun-5ffb', version=111, file='OmicsSomaticMutationsMatrixDamaging')
OmicsSomaticMutationsMatrixDriver = tc.get(name='internal-22q2dryrun-5ffb', version=111, file='OmicsSomaticMutationsMatrixDriver')
OmicsSomaticMutationsMatrixHotspot = tc.get(name='internal-22q2dryrun-5ffb', version=111, file='OmicsSomaticMutationsMatrixHotspot')

In [ ]:
OmicsSomaticMutationsMatrixDamaging = OmicsSomaticMutationsMatrixDamaging.fillna(0).astype(int)
OmicsSomaticMutationsMatrixDriver = OmicsSomaticMutationsMatrixDriver.fillna(0).astype(int)
OmicsSomaticMutationsMatrixHotspot = OmicsSomaticMutationsMatrixHotspot.fillna(0).astype(int)

In [ ]:
OmicsSomaticMutationsMatrixHotspot

In [ ]:
OmicsSomaticMutationsMatrixDamaging.to_csv("OmicsSomaticMutationsMatrixDamaging.csv")
OmicsSomaticMutationsMatrixDriver.to_csv("OmicsSomaticMutationsMatrixDriver.csv")
OmicsSomaticMutationsMatrixHotspot.to_csv("OmicsSomaticMutationsMatrixHotspot.csv")

In [ ]:
tc.update_dataset(
    changes_description="convert nans to 0s",
    dataset_permaname="internal-22q2dryrun-5ffb",
    upload_files=[            
        {
            "path": "OmicsSomaticMutationsMatrixDamaging.csv",
            "format": "NumericMatrixCSV",
            "encoding": "utf-8",
        },
        {
            "path": "OmicsSomaticMutationsMatrixDriver.csv",
            "format": "NumericMatrixCSV",
            "encoding": "utf-8",
        },
        {
            "path": "OmicsSomaticMutationsMatrixHotspot.csv",
            "format": "NumericMatrixCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
CCLE_mutations_bool_hotspot = tc.get(name='internal-22q2-097a', version=16, file='CCLE_mutations_bool_hotspot')


In [ ]:
import numpy as np

CCLE_mutations_bool_hotspot.replace(0,np.nan).dropna(axis="columns", how="all")

In [ ]:
CCLE_mutations_bool_hotspot.sum().value_counts()

## subset Q4 mutation data for Lisa and Kirsty

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsSomaticMutations = tc.get(name='internal-22q4-56d4', version=21, file='OmicsSomaticMutations')

In [ ]:
lisa_mutations = OmicsSomaticMutations[OmicsSomaticMutations.HugoSymbol.isin(lisa_genes)]

In [ ]:
lisa_mutations.to_csv("mutation_subset_by_genes.csv", index=False)

In [ ]:
kirsty_lines = ["ACH-000529",
"ACH-001018",
"ACH-001087",
"ACH-001169",
"ACH-001179",
"ACH-001180",
"ACH-001181",
"ACH-001187",
"ACH-001357",
"ACH-001453",
"ACH-001767",
"ACH-001827",
"ACH-001854",
"ACH-002210",
"ACH-002247",
"ACH-002270",
"ACH-002319",
"ACH-002321",
"ACH-002327",
"ACH-002336",
"ACH-002558",
"ACH-002591",
"ACH-002592",
"ACH-002593",
"ACH-002609",
"ACH-002610",
"ACH-002611",
"ACH-002623",
"ACH-002624",
"ACH-002625",
"ACH-002712",
"ACH-003016",
"ACH-003020",
"ACH-003062",
"ACH-003068",
"ACH-003104",
"ACH-003117",
"ACH-003118"]

In [ ]:
kirsty_mutations = OmicsSomaticMutations[OmicsSomaticMutations.DepMap_ID.isin(kirsty_lines)]

In [ ]:
kirsty_mutations.to_csv("mutation_subset_by_lines.csv", index=False)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

a = tc.create_dataset(
    "22Q4_mutation_subset",
    dataset_description="subset of 22Q4 mutation data (NOT QC'ED) for Kirsty and Lisa",
    upload_files=[
        {
            "path": "mutation_subset_by_lines.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
        {
            "path": "mutation_subset_by_genes.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd",
)

In [ ]:
mutation_subset_by_lines = tc.get(name='22q4-mutation-subset-a22e', version=1, file='mutation_subset_by_lines')


In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
fingerprint_lod_matrix = tc.get(name='ccle-bam-fingerprints-6f30', file='fingerprint_lod_matrix')

In [ ]:
fingerprint_lod_matrix.loc[['CDS-wkBjwZ', 'CDS-7wtGGb'], ['CDS-wkBjwZ', 'CDS-7wtGGb']]

In [ ]:
OmicsDefaultModelConditionProfiles = tc.get(name='internal-22q4-56d4', version=68, file='OmicsDefaultModelConditionProfiles')
OmicsDefaultModelConditionProfiles_old = tc.get(name='internal-22q4-56d4', version=57, file='OmicsDefaultModelConditionProfiles')


In [ ]:
[i for i in set(OmicsDefaultModelConditionProfiles.ProfileID) if i not in set(OmicsDefaultModelConditionProfiles_old.ProfileID)]

In [ ]:
OmicsSomaticMutationsMatrixDamaging_q4 = tc.get(name='internal-22q4-56d4', version=69, file='OmicsSomaticMutationsMatrixDamaging')
OmicsSomaticMutationsMatrixDamaging_dryrun = tc.get(name='internal-22q2dryrun-5ffb', version=113, file='OmicsSomaticMutationsMatrixDamaging')


In [ ]:
OmicsSomaticMutationsMatrixDamaging_q4 = OmicsSomaticMutationsMatrixDamaging_q4[[c for c in OmicsSomaticMutationsMatrixDamaging_q4.columns if c in set(OmicsSomaticMutationsMatrixDamaging_dryrun.columns)]]


In [ ]:
OmicsSomaticMutationsMatrixDamaging_dryrun = OmicsSomaticMutationsMatrixDamaging_dryrun[[c for c in OmicsSomaticMutationsMatrixDamaging_dryrun.columns if c in set(OmicsSomaticMutationsMatrixDamaging_q4.columns)]]


In [ ]:
OmicsSomaticMutationsMatrixDamaging_q4 = OmicsSomaticMutationsMatrixDamaging_q4.loc[[i for i in OmicsSomaticMutationsMatrixDamaging_q4.index if i in set(OmicsSomaticMutationsMatrixDamaging_dryrun.index)]]

In [ ]:
OmicsSomaticMutationsMatrixDamaging_dryrun = OmicsSomaticMutationsMatrixDamaging_dryrun.loc[[i for i in OmicsSomaticMutationsMatrixDamaging_dryrun.index if i in set(OmicsSomaticMutationsMatrixDamaging_q4.index)]]

In [ ]:
OmicsSomaticMutationsMatrixDamaging_q4.sort_index(inplace=True)
OmicsSomaticMutationsMatrixDamaging_dryrun.sort_index(inplace=True)

In [ ]:
set(OmicsSomaticMutationsMatrixDamaging_q4.columns) == set(OmicsSomaticMutationsMatrixDamaging_dryrun.columns)

In [ ]:
OmicsDefaultModelConditionProfiles = tc.get(name='internal-22q4-56d4', version=69, file='OmicsDefaultModelConditionProfiles')
OmicsDefaultModelProfiles = tc.get(name='internal-22q4-56d4', version=69, file='OmicsDefaultModelProfiles')


In [ ]:
OmicsDefaultModelConditionProfiles[OmicsDefaultModelConditionProfiles.ModelConditionID == "MC-001227-21hV"]

In [ ]:
proteinCoding_genes_tpm_logp1_withReplicates = tc.get(name='expression-d035', version=29, file='proteinCoding_genes_tpm_logp1_withReplicates')


In [ ]:
"CDS-vvQhsI" in proteinCoding_genes_tpm_logp1_withReplicates.index